### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Portuguese"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Portuguese
Lang Id = 5
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic,https://www.youtube.com/watch?v=RIdNWADMmic&t=...
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs,https://www.youtube.com/watch?v=Wb5154ZxiHs&t=...
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE,https://www.youtube.com/watch?v=98B7hGXu9YE&t=...
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo,https://www.youtube.com/watch?v=TQTjOYzo9bo&t=...
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves,https://www.youtube.com/watch?v=TKPATf95Ves&t=...
...,...,...,...,...,...,...
195,mas como,413,415,mas como,nixBhyA3sog,https://www.youtube.com/watch?v=nixBhyA3sog&t=...
196,não está,1176,1177,não está,TX_RyDN6YFc,https://www.youtube.com/watch?v=TX_RyDN6YFc&t=...
197,mas para,487,493,mas para,2pYNXoeyjxM,https://www.youtube.com/watch?v=2pYNXoeyjxM&t=...
198,mas para,428,437,mas para,s9U6k3BEPmw,https://www.youtube.com/watch?v=s9U6k3BEPmw&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,o seu tempo porque eu não tenho nada de dinhei...,2049,2056,o seu tempo porque eu não tenho nada de dinhei...,2wA_t6MXdcU,https://www.youtube.com/watch?v=2wA_t6MXdcU&t=...
1,quer para nós vou fazer uma coisa que ela não ...,84,89,quer para nós vou fazer uma coisa que ela não ...,cm9OuklSuZ4,https://www.youtube.com/watch?v=cm9OuklSuZ4&t=84s
2,você pode fazer assim um por favor faz isso pa...,1126,1132,você pode fazer assim um por favor faz isso pa...,5HsmMvokixI,https://www.youtube.com/watch?v=5HsmMvokixI&t=...
3,de tempo ele já foi a coisa mais espera agora ...,809,814,um lapso de tempo ele já foi a coisa mais espe...,GcYPFyk5NAk,https://www.youtube.com/watch?v=GcYPFyk5NAk&t=...
4,com a minha mulher é como é porque eu tenho qu...,2926,2930,a relação com a minha mulher é como é porque e...,61FQdHgTyhA,https://www.youtube.com/watch?v=61FQdHgTyhA&t=...
...,...,...,...,...,...,...
195,tem as coisas como já são a vida como ela é,129,135,tem as coisas como já são a vida como ela é,sblaVJJpLnM,https://www.youtube.com/watch?v=sblaVJJpLnM&t=...
196,e vão ficar por quê porque tem esse tipo de,526,530,tão aí vivas e vão ficar por quê porque tem es...,y6kH0n06hQ0,https://www.youtube.com/watch?v=y6kH0n06hQ0&t=...
197,foi quando minha mãe também estava por aqui,494,496,foi quando minha mãe também estava por aqui,c8AnL2XA1mA,https://www.youtube.com/watch?v=c8AnL2XA1mA&t=...
198,vamos ver se vai ser essa ou se vai ser uma,656,659,vamos ver se vai ser essa ou se vai ser uma pr...,Q3Vdd0O8qgs,https://www.youtube.com/watch?v=Q3Vdd0O8qgs&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,de qualquer forma eu tenho certeza que você ou...,58,68,de qualquer forma eu tenho certeza que você ou...,Ng85w0W2lRo,https://www.youtube.com/watch?v=Ng85w0W2lRo&t=58s
1,e ela dá a ideia de que ah eu não tenho nada a...,1189,1197,e ela dá a ideia de que ah eu não tenho nada a...,5yYk7YTnKdo,https://www.youtube.com/watch?v=5yYk7YTnKdo&t=...
2,nosso bebé pela primeira vez nem sempre aconte...,152,159,nosso bebé pela primeira vez nem sempre aconte...,qpm0lpb0iuI,https://www.youtube.com/watch?v=qpm0lpb0iuI&t=...
3,então de alguma forma eu tinha que fazer isso ...,122,130,então de alguma forma eu tinha que fazer isso ...,Eq7Ntuz42Vo,https://www.youtube.com/watch?v=Eq7Ntuz42Vo&t=...
4,do ponto de vista assim é que a gente vai até ...,581,587,depende do ponto de vista assim é que a gente ...,ASK6PbmZNq4,https://www.youtube.com/watch?v=ASK6PbmZNq4&t=...
...,...,...,...,...,...,...
195,e talvez olhe uma vez ao dia ou uma vez a cada...,349,357,e talvez olhe uma vez ao dia ou uma vez a cada...,LZZnpnxDSSI,https://www.youtube.com/watch?v=LZZnpnxDSSI&t=...
196,consegue ver aquela rua todos os dias aquele p...,1621,1627,consegue ver aquela rua todos os dias aquele p...,UMLaEo_cp58,https://www.youtube.com/watch?v=UMLaEo_cp58&t=...
197,você dizer coisas e ela fica mais para mãe as ...,154,161,você dizer coisas e ela fica mais para mãe as ...,eCSw89UGbt0,https://www.youtube.com/watch?v=eCSw89UGbt0&t=...
198,está mais feliz é porque alguma coisa precisa ...,618,626,está mais feliz é porque alguma coisa precisa ...,PO0F_to_bQo,https://www.youtube.com/watch?v=PO0F_to_bQo&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
1,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
2,vou explicar isso para você está é uma coisa b...,278,286,vou explicar isso para você está é uma coisa b...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
3,o japão finalmente tinha se tornado um exemplo...,369,377,o japão finalmente tinha se tornado um exemplo...,0ZQCwWbPbN4,https://www.youtube.com/watch?v=0ZQCwWbPbN4&t=...
4,nesse caso você vai colocar em cima do dedo as...,621,630,nesse caso você vai colocar em cima do dedo as...,Z7_CtSVJBmY,https://www.youtube.com/watch?v=Z7_CtSVJBmY&t=...
...,...,...,...,...,...,...
195,sem água hoje eu vim para cá em setembro então...,641,649,sem água hoje eu vim para cá em setembro então...,Nz5yW6fmyZ0,https://www.youtube.com/watch?v=Nz5yW6fmyZ0&t=...
196,você quer ter uma uma rede sem lojas não porqu...,1745,1752,você quer ter uma uma rede sem lojas não porqu...,nPzKJ5VQwj0,https://www.youtube.com/watch?v=nPzKJ5VQwj0&t=...
197,almoço nossa julia verdade tem razão se eu fiz...,1060,1066,almoço nossa julia verdade tem razão se eu fiz...,OifnBPkVwGc,https://www.youtube.com/watch?v=OifnBPkVwGc&t=...
198,medo um receio não sei isso me lembra me lembr...,1920,1932,medo um receio não sei isso me lembra me lembr...,nCBEJhCwqJQ,https://www.youtube.com/watch?v=nCBEJhCwqJQ&t=...


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
2,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
3,adoraria ter o problema é o valor dele né pess...,302,314,adoraria ter o problema é o valor dele né pess...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
4,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
...,...,...,...,...,...,...
195,o brasil vai para trás mas é real isso vamos o...,94,103,o brasil vai para trás mas é real isso vamos o...,fa4R_oqCjbM,https://www.youtube.com/watch?v=fa4R_oqCjbM&t=94s
196,na verdade para se falar mais nesse assunto é ...,1843,1850,na verdade para se falar mais nesse assunto é ...,hWszDAo_Ifo,https://www.youtube.com/watch?v=hWszDAo_Ifo&t=...
197,alguma coisa cai em cima dele ou então é duran...,122,130,alguma coisa cai em cima dele ou então é duran...,Xwr_9M4VBeg,https://www.youtube.com/watch?v=Xwr_9M4VBeg&t=...
198,porque o real sempre está no presente esse ret...,742,751,porque o real sempre está no presente esse ret...,Wcdol8o3Foc,https://www.youtube.com/watch?v=Wcdol8o3Foc&t=...


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o,https://www.youtube.com/watch?v=BampggOg28o&t=...
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE,https://www.youtube.com/watch?v=mAHS9bghaLE&t=...
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ,https://www.youtube.com/watch?v=fvtbRqPsTtQ&t=40s
...,...,...,...,...,...,...
195,se relacionar com todos os profissionais envol...,90,98,se relacionar com todos os profissionais envol...,ayHBi4tp3dI,https://www.youtube.com/watch?v=ayHBi4tp3dI&t=90s
196,mamã também vou ter mais dificuldade em adorme...,252,259,mamã também vou ter mais dificuldade em adorme...,Ff1YZJwYz0s,https://www.youtube.com/watch?v=Ff1YZJwYz0s&t=...
197,essas pessoas ficam ainda mais vulneráveis poi...,94,100,essas pessoas ficam ainda mais vulneráveis poi...,usCDzMSAmaY,https://www.youtube.com/watch?v=usCDzMSAmaY&t=94s
198,o seguro do carro também extremamente importan...,231,237,o seguro do carro também extremamente importan...,yhiDM4WYvqQ,https://www.youtube.com/watch?v=yhiDM4WYvqQ&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,que você pode estar utilizando ai em locais re...,1237,1252,que você pode estar utilizando ai em locais re...,DB31ARhbwAM,https://www.youtube.com/watch?v=DB31ARhbwAM&t=...
2,mas também com os nossos parceiros da indústri...,639,650,mas também com os nossos parceiros da indústri...,ELSjYhhLR9g,https://www.youtube.com/watch?v=ELSjYhhLR9g&t=...
3,porque os velhos espinhos e folhas estavam se ...,553,566,porque os velhos espinhos e folhas estavam se ...,65QGNVrSKwI,https://www.youtube.com/watch?v=65QGNVrSKwI&t=...
4,recentemente eu estou dando foco nos assuntos ...,32,44,recentemente eu estou dando foco nos assuntos ...,aEPd1q1ibbE,https://www.youtube.com/watch?v=aEPd1q1ibbE&t=32s
...,...,...,...,...,...,...
195,um prazer te escutar sigamos nos encontramos a...,3128,3136,um prazer te escutar sigamos nos encontramos a...,PgkDEe4cWJ0,https://www.youtube.com/watch?v=PgkDEe4cWJ0&t=...
196,embora complicações graves sejam raras laceraç...,674,684,embora complicações graves sejam raras laceraç...,_vGSPbYgvTI,https://www.youtube.com/watch?v=_vGSPbYgvTI&t=...
197,mas definitivamente é um passeio muito especia...,426,435,mas definitivamente é um passeio muito especia...,czqaKhnUgko,https://www.youtube.com/watch?v=czqaKhnUgko&t=...
198,pra agendar uma visita ou saber mais informaçõ...,224,231,pra agendar uma visita ou saber mais informaçõ...,mlj232DASDw,https://www.youtube.com/watch?v=mlj232DASDw&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [16]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [17]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [18]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [19]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [20]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [21]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [22]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [23]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [24]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1 Level", "videos":A1_data_list}
#data_A1

##### A2 Level

In [25]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [26]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [27]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [28]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2 Level", "videos":A2_data_list}
#data_A2

##### B1 Level

In [29]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [30]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [31]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [32]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1 Level", "videos":B1_data_list}
#data_B1

##### B2 Level

In [33]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [34]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [35]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [36]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2 Level", "videos":B2_data_list}
#data_B2

##### C1 Level

In [37]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [38]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [39]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [40]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1 Level", "videos":C1_data_list}
#data_C1

##### C2 Level

In [41]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [42]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [43]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [44]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2 Level", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [45]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [46]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1670231368
  nanos: 853522000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass